In [10]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [11]:
import pandas as pd

In [12]:
import sentencepiece as spm

In [13]:
#Đọc dữ liệu
review_data = pd.read_pickle('/media/zero/Data/Dropbox/metadata/foody review/foody review full.pkl')

In [29]:
review_data = review_data.iloc[0:100]

In [15]:
review_data['label'] = review_data['score'].map(lambda x: 'pos' if x > 5 else 'neg')

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(review_data[['review_content']], 
                                                    review_data[['label']], 
                                                    test_size=0.2, random_state=2018)

In [9]:
#write to text file

with open("/media/zero/Data/Dropbox/Data science project/Foody review/Deep learning/review.txt", "a") as f:
    for t in x_train.review_content.values:
        f.write(t.replace('\n', '').lower()); f.write('\n')
    f.close()

In [28]:
!cd /home/zero/sentencepiece && spm_train --input=review.txt --model_prefix=review_model --vocab_size=10000

unigram_model_trainer.cc(494) LOG(INFO) Starts training with : 
input: "review.txt"
model_prefix: "review_model"
model_type: UNIGRAM
vocab_size: 10000
character_coverage: 0.9995
input_sentence_size: 10000000
mining_sentence_size: 2000000
training_sentence_size: 10000000
seed_sentencepiece_size: 1000000
shrinking_factor: 0.75
num_threads: 16
num_sub_iterations: 2
max_sentencepiece_length: 16
split_by_unicode_script: true
split_by_whitespace: true

trainer_interface.cc(109) LOG(INFO) Loading corpus: review.txt
trainer_interface.cc(126) LOG(INFO) Loading: ▁the▁one▁coffee▁&▁bakery▁-▁hùng▁vươngnằm▁ở▁vị▁trí▁trung▁tâm▁thành▁phố,▁không▁gian▁rộng,▁view▁nhìn▁ra▁đẹp.▁các▁ngày▁lễ▁thường▁có▁hoạt▁động▁kèm▁theo.▁thường▁có▁cập▁nhật▁món▁mới.▁kem▁topping▁làm▁rất▁ngon,▁béo▁vừa▁đủ.▁mattcha▁thì▁lạt.▁coffee▁nghệ▁thuật▁ý▁tuyệt▁vời,▁đậm▁và▁thơm.▁nhân▁viên▁đẹp,▁nhiệt▁tình.▁chỗ▁bỏ▁xe▁thoáng▁và▁có▁nhân▁viên.▁cách▁bày▁trí▁bàn▁ghế▁đẹp,▁đa▁dạng.	size=0
trainer_interface.cc(132) LOG(WARNING) Empty string found. remo

In [18]:
sp = spm.SentencePieceProcessor()
sp.Load("/home/zero/sentencepiece/review_model.model")

True

In [19]:
def subword_tok(sentence):
    tok = [t for t in sp.EncodeAsPieces(sentence.lower())]
    return tok

In [20]:
TEXT = data.Field(lower=True, tokenize=subword_tok)

In [21]:
bs=64; bptt=70

In [22]:
bs=64; bptt=70
md = LanguageModelData.from_dataframes(path = '', field = TEXT,
                                       col = 'review_content',
                                       train_df = x_train, val_df = x_test, test_df = x_test,
                                       bs=bs, bptt=bptt, min_freq=5)

In [23]:
pickle.dump(TEXT, open('model/TEXT.pkl','wb'))

In [31]:
#train
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [32]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
                       dropouti=0.05, dropout=0.05, 
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [33]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss   
    0      6.618516   6.206409  
    1      6.264497   5.766661  
    2      6.081293   5.714104  
    3      5.986367   5.683962  
    4      5.926641   5.66617   
    5      5.883131   5.660097  
    6      5.851362   5.662056  
    7      5.829514   5.635532  
    8      5.801358   5.526217  
    9      5.766962   5.489103  
    10     5.732366   5.45989   
    11     5.694668   5.393836  
    12     5.657704   5.370064  
    13     5.623039   5.341275  
    14     5.594877   5.343325  



[5.3433247]

In [34]:
learner.save_encoder('lang_model')

In [35]:
learner.load_encoder('lang_model')

In [36]:
m=learner.model

In [37]:
m=learner.model
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [94]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs


TypeError: 'int' object is not callable

In [89]:
#play around
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))

    print('...')


. So, it wasn't quite was I was expecting, but I really liked it anyway! The best 



NameError: name 'res' is not defined

In [ ]:
#add examples

In [38]:
class DataFrameDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, text_col, label_col, dfs, **kwargs):
        fields = [("text", text_field), ("label", label_field)]
        examples = []
                
        for key, df in dfs.items():
            for i, row in df.iterrows():
                text = row[text_col]
                label = row[label_col]
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, path, col, label, train, valid=None, test=None, **kwargs):
        dfs = {'train': train}

        if valid is not None:
            dfs['valid'] = valid
            has_validation = 'valid'
        else:
            has_validation = None
        if test is not None:
            dfs['test'] = test
            has_test = 'test'
        else:
            has_test = None

        return super().splits(path, 
                                text_field=text_field, label_field=label_field, 
                                text_col=col, label_col=label, 
                                train='train', validation=has_validation, test=has_test,  
                                dfs=dfs, **kwargs)

In [39]:

LABEL = data.Field(sequential=False)

splits = DataFrameDataset.splits(
                text_field=TEXT, 
                label_field=LABEL,
                path='',
                col="review_content",
                label="label",
                train=pd.concat((x_train[['review_content']], y_train[['label']]), axis = 1), 
                valid=pd.concat((x_test[['review_content']], y_test[['label']]), axis = 1))

In [40]:
splits[0].examples[16]
t = splits[0].examples[16]
t.label, sp.DecodePieces((t.text[:16]))

('pos', 'quán nằm gần chợ đà lạt \ntầng trên có khách sạn 🏨 giá')

In [41]:
#create model

In [42]:
PATH = ''
md2 = TextData.from_splits(PATH, splits, bs)

In [43]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'lang_model')

In [44]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [45]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 0/7 [00:00<?, ?it/s]


AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx